In [ ]:
import yaml
import numpy as np
import ultraplot as uplt

from mattspy.twoptdata import TwoPtData

dmap = TwoPtData.read_des_twopoint("3x2pt_map.fits")

# read data, cut to only <250 arcmin and remove wtheta cross corr
d = TwoPtData.read_des_twopoint("3x2pt_2025-06-18-09h_UNBLINDED.fits")
d = d.cut_angle(-1, 250).cut_wtheta_crosscorr()

# get the correct cov matrix w/ point mass marg. and rebuild with it
icov = np.load("invcov_with_pm_1000.npy")
cov = np.linalg.inv(icov)
d = d.rebuild_only_unmasked().replace_full_cov(cov)

# apply analysis cuts
with open("cuts.yml") as fp:
    cut_yml = yaml.load(fp.read(), Loader=yaml.SafeLoader)

d = d.cut_cosmosis(cut_yml['cuts_linb_32_965']['params'])

In [ ]:
from itertools import combinations

vals = ["l2", "s3", "s4", "l5", "l6"]

res = {}

res[None] = d.chi2_stats(dmap, 18)

for i in range(len(vals)):
    for c in combinations(vals, i+1):
        _d = d
        for si in c:
            _d = _d.cut_component(si[0], si[1])

        key = "_".join([str(si) for si in c])
        res[key] = _d.chi2_stats(dmap, 18)

In [ ]:
print(yaml.dump(res, indent=2, default_flow_style=False))

In [ ]:
import numpy as np

dnl = d.cut_component("l", 2)

res = {}
for frac in np.linspace(0.0, 0.15, 10):
    _d = dnl.increase_cov_fractionally(frac)
    res[f"{frac:0.3}"] = _d.chi2_stats(dmap, 18)

print(yaml.dump(res, indent=2, default_flow_style=False))

In [ ]:
fig, axs = uplt.subplots()

fracs = []
nsigs = []
for k in res:
    fracs.append(float(k))
    nsigs.append(res[k]["nsigma"])


axs.plot(fracs, nsigs)
axs.format(xlabel="fractional increase in std", ylabel="# of sigmas", title="3x2pt w/o lens bin 2")